In [1]:
import os
import logging
import random

# Load everything

In [2]:
from dask.distributed import Client, LocalCluster

In [3]:
dask_logger = logging.getLogger("distributed.utils_perf")
dask_logger.setLevel(logging.ERROR)

In [4]:
dask_client = Client(LocalCluster(
    n_workers=10,
    threads_per_worker=1))

2022-07-14 14:01:54,825 - distributed.diskutils - INFO - Found stale lock file and directory '/home/eeisenst/workspace/compassionai/garland/notebooks/dask-worker-space/worker-08uccyfm', purging
2022-07-14 14:01:54,826 - distributed.diskutils - INFO - Found stale lock file and directory '/home/eeisenst/workspace/compassionai/garland/notebooks/dask-worker-space/worker-g2j6osib', purging
2022-07-14 14:01:54,826 - distributed.diskutils - INFO - Found stale lock file and directory '/home/eeisenst/workspace/compassionai/garland/notebooks/dask-worker-space/worker-ec6x9y4c', purging
2022-07-14 14:01:54,826 - distributed.diskutils - INFO - Found stale lock file and directory '/home/eeisenst/workspace/compassionai/garland/notebooks/dask-worker-space/worker-fnr3iqje', purging
2022-07-14 14:01:54,826 - distributed.diskutils - INFO - Found stale lock file and directory '/home/eeisenst/workspace/compassionai/garland/notebooks/dask-worker-space/worker-d4pdf070', purging
2022-07-14 14:01:54,826 - dist

In [5]:
dask_client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 10,Total memory: 31.17 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42707,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 10
Started: Just now,Total memory: 31.17 GiB
Comm: tcp://127.0.0.1:46735,Total threads: 1
Dashboard: http://127.0.0.1:37863/status,Memory: 3.12 GiB
Nanny: tcp://127.0.0.1:36297,


## Data

In [6]:
from cai_common.data import ParallelTMXLoader

In [7]:
ParallelTMXLoader.data_glob = os.path.join("raw_datasets/84000-parallel-sentences", "*.tmx")

In [8]:
parallel_df = ParallelTMXLoader() \
    .apply_markup() \
    .clean_bad_chars() \
    .dataframe

In [9]:
parallel_df = parallel_df[["tohoku", "tibetan", "english"]]
parallel_df = parallel_df[parallel_df.tohoku.str.isnumeric()]

In [10]:
parallel_df["tohoku"] = parallel_df.tohoku.fillna(-1).astype(int)

## Config (needed for preprocessing)

In [11]:
from hydra import initialize, compose
from hydra.utils import instantiate
from omegaconf import OmegaConf

In [12]:
initialize(version_base="1.2", config_path="../cai_garland/data/parallel_dataset_prep.config")
cfg = compose(config_name="config")

In [13]:
# print(OmegaConf.to_yaml(cfg))

# Prepare data

## Select relevant data

In [14]:
tohoku_numbers = [298, 312, 86, 171, 245]

In [15]:
test_df = parallel_df[parallel_df.tohoku.isin(tohoku_numbers)][["tohoku", "tibetan", "english"]].compute()

In [16]:
test_flat_data = test_df.to_dict(orient="records")

## Preprocess

In [17]:
from cai_garland.data.parallel_dataset_prep import _preprocess_flat_data
from cai_garland.utils.str_processors import ProcessorSymbolCleaningJSON

In [18]:
ProcessorSymbolCleaningJSON.base_dir = os.path.dirname("../cai_garland/data/")

In [19]:
test_flat_data_proced = _preprocess_flat_data(test_flat_data, cfg)

In [20]:
test_flat_data = [
    {
        'tibetan': proced['tibetan'],
        'english': proced['english'],
        'tohoku': orig['tohoku']
    }
    for orig, proced in zip(test_flat_data, test_flat_data_proced)
]

## Postprocess

In [21]:
# Tibetan
bo_lines = [datum['tibetan'] for datum in test_flat_data]
for processor in cfg.output.postprocessing.source_lang:
    processor = instantiate(processor)
    bo_lines = [processor(line) for line in bo_lines]

In [22]:
# English
en_lines = [datum['english'] for datum in test_flat_data]
for processor in cfg.output.postprocessing.target_lang:
    processor = instantiate(processor)
    en_lines = [processor(line) for line in en_lines]

In [23]:
en_lines[:10]

['the noble great vehicle sutra the sections of dharma',
 'homage to all buddhas and bodhisattvas.',
 'thus did i hear at one time.',
 'the blessed one was dwelling in nyagrodha park, in kapilavastu, together with a great sangha of five hundred monks.',
 'at one point venerable sariputra, venerable mahamaudgalyayana, venerable mahakasyapa, venerable ananda, and venerable rahula all went to see the blessed one.',
 'they prostrated to him, placing their heads at his feet, and then sat to one side.',
 'sitting there, venerable sariputra asked the blessed one,',
 'but what is the extent of those sections of the dharma to which the thus-gone one refers using that expression?',
 'how many such sections are there?',
 'the blessed one replied to venerable sariputra,']

# Translate

In [24]:
from tqdm.auto import tqdm

In [25]:
from cai_garland.utils.translator import Translator

In [26]:
translator = Translator(os.path.join("olive-cormorant-bart", "base-base"))
translator.num_beams = 10
translator.cuda()

In [27]:
en_translations = [translator.translate(line) for line in tqdm(bo_lines)]

  0%|          | 0/702 [00:00<?, ?it/s]

In [28]:
random.seed(4)
for translation, label in random.choices(list(zip(en_translations, en_lines)), k=10):
    print(translation)
    print(label)
    print()

a woman, even if she is the beauty of the land, sixteen years old, an excellent woman, with a beautiful form, who is lovely to behold, with delightful full breasts, adorned with jewelry, youthful, pleasant to touch, and free of disease.
a woman, even if she is the beauty of the land, sixteen years old, an excellent woman, with a beautiful form, who is lovely to behold, with delightful full breasts, adorned with jewelry, youthful, pleasant to touch, and free of disease.

they cause ones virtuous conduct to degenerate, and therefore we speak of eyes.
they cause ones virtuous conduct to degenerate, and therefore we speak of eyes.

temples also will be destroyed, and
temples also will be destroyed, and

the sense pleasures are not worthy of being looked upon.
desires too are not worthy of being looked upon.

they think of women as women.
they think of women as women.

make it six fingers in breadth and two fingers in height.
make it six fingers in breadth and two fingers in height.

this, 

# Score

In [29]:
from datasets import load_metric

In [30]:
metric = load_metric("sacrebleu")

In [31]:
by_tohoku = {}
for orig, translation, label in zip(test_flat_data, en_translations, en_lines):
    by_tohoku[orig['tohoku']] = by_tohoku.get(orig['tohoku'], {'preds': [], 'labels': []})
    by_tohoku[orig['tohoku']]['preds'].append(translation)
    by_tohoku[orig['tohoku']]['labels'].append([label])

In [32]:
for tohoku, data in by_tohoku.items():
    bleu = metric.compute(predictions=data['preds'], references=data['labels'])['score']
    print(f"{tohoku}: {bleu}")

245: 85.19516406831191
298: 92.5699849927083
312: 86.37222290801779
171: 87.98466415543461
